In [2]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

## Q1. Downloading the data

In [7]:
tripdata_jan21 = pd.read_parquet('yellow_tripdata_2023-01.parquet')
tripdata_feb21 = pd.read_parquet('yellow_tripdata_2023-02.parquet')

In [8]:
tripdata_jan21.shape

(3066766, 19)

## Q2. Computing duration

In [9]:
tripdata_jan21

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.30,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.90,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.90,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.10,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.40,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066761,2,2023-01-31 23:58:34,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,0,15.80,0.00,0.5,3.96,0.0,1.0,23.76,NaN,NaN
3066762,2,2023-01-31 23:31:09,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,0,22.43,0.00,0.5,2.64,0.0,1.0,29.07,NaN,NaN
3066763,2,2023-01-31 23:01:05,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,0,17.61,0.00,0.5,5.32,0.0,1.0,26.93,NaN,NaN
3066764,2,2023-01-31 23:40:00,2023-01-31 23:53:00,NaN,3.15,NaN,None,230,79,0,18.15,0.00,0.5,4.43,0.0,1.0,26.58,NaN,NaN


In [11]:
tripdata_jan21.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [12]:
tripdata_jan21['duration'] = tripdata_jan21.apply(lambda x: pd.to_datetime(x.tpep_dropoff_datetime, infer_datetime_format=True) - pd.to_datetime(x.tpep_pickup_datetime, infer_datetime_format=True), axis = 1)
tripdata_jan21['duration'] = tripdata_jan21['duration'].astype('timedelta64[s]')/60

In [13]:
tripdata_jan21['duration'].mean()

15.668995167330452

In [14]:
tripdata_jan21['duration'].describe()

count    3.066766e+06
mean     1.566900e+01
std      4.259435e+01
min     -2.920000e+01
25%      7.116667e+00
50%      1.151667e+01
75%      1.830000e+01
max      1.002918e+04
Name: duration, dtype: float64

In [15]:
tripdata_jan21_no_outliers = tripdata_jan21.query('(duration >= 1) & (duration <= 60)')

print('We removed ' + str(tripdata_jan21.shape[0] - tripdata_jan21_no_outliers.shape[0]) + ' records, by filtering for duration between 1 and 60 mins inclusive.')

We removed 57593 records, by filtering for duration between 1 and 60 mins inclusive.


## Q3. Missing values

In [124]:
na_columns = ['PUlocationID','DOlocationID']

for col in na_columns:
    tripdata_jan21_no_outliers[col].fillna(-1, inplace = True)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6383: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [16]:
missing_value_count = tripdata_jan21_no_outliers.query('PUlocationID == -1').shape[0]
missing_value_fraction = missing_value_count/ tripdata_jan21_no_outliers.shape[0] * 100
missing_value_fraction

UndefinedVariableError: name 'PUlocationID' is not defined

## Q4. One-hot encoding

One advantage of using DictVectorizer instead of get_dummies is that it supports sparse matrix output. Thus, if we have a large number of features, DictVectorizer will be a good choice:

- Reference: https://towardsdatascience.com/encoding-categorical-features-21a2651a065c

In [17]:
train_dicts = tripdata_jan21_no_outliers[na_columns].astype(str).to_dict(orient='records')   #convert df to dict

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

NameError: name 'na_columns' is not defined

In [18]:
X_train

NameError: name 'X_train' is not defined

In [19]:
X_train.shape[1]

NameError: name 'X_train' is not defined

525 columns, each PUlocationID or DOlocationID becomes a column/feature

## Q5. Training a model

In [20]:
y_train = tripdata_jan21_no_outliers['duration'].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

NameError: name 'X_train' is not defined

## Q6. Evaluating the model

In [21]:
#Repeat same preprocessing for validation set
tripdata_feb21['duration'] = tripdata_feb21.apply(lambda x: pd.to_datetime(x.dropOff_datetime, infer_datetime_format=True) - pd.to_datetime(x.pickup_datetime, infer_datetime_format=True), axis = 1)
tripdata_feb21['duration'] = tripdata_feb21['duration'].astype('timedelta64[s]')/60

tripdata_feb21_no_outliers = tripdata_feb21.query('(duration >= 1) & (duration <= 60)')

for col in na_columns:
    tripdata_feb21_no_outliers[col].fillna(-1, inplace = True)

AttributeError: 'Series' object has no attribute 'dropOff_datetime'

In [ ]:
val_dicts = tripdata_feb21_no_outliers[na_columns].astype(str).to_dict(orient='records')

X_val = dv.transform(val_dicts)           #Apply previous dictvectorizer on validation set
y_val = tripdata_feb21_no_outliers['duration'].values

y_pred_val = lr.predict(X_val)

mean_squared_error(y_val, y_pred_val, squared=False)

11.014283196111764